In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyntcloud import PyntCloud

In [ ]:
class VoxelDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.from_numpy(sample).float()

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, padding=0),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv3d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
directory = "abc-dataset-ply/"
files = sorted([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])
dataset = []

def convert_to_binvox(path):
    point_cloud = np.loadtxt(path, skiprows=12)[:, 0:3]
    df = pd.DataFrame(data=point_cloud, columns=['x','y','z'])
    cloud = PyntCloud(df)
    voxelgrid_id = cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)
    voxelgrid = cloud.structures[voxelgrid_id]
    Binary_voxel_array = voxelgrid.get_feature_vector(mode="binary")
    dataset.append(Binary_voxel_array)

for i in files:
    path = os.path.join(directory, i)
    convert_to_binvox(path)

In [ ]:
train_data = dataset[:35]
test_data = dataset[35:]

# Create PyTorch datasets and dataloaders
train_dataset = VoxelDataset(train_data)
test_dataset = VoxelDataset(test_data)

# Determine the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model, optimizer, and loss function
autoencoder = Autoencoder().to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)  # Move data to the GPU
        outputs = autoencoder(data.unsqueeze(1).to(device))
        loss = criterion(outputs, data.unsqueeze(1).to(device))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
autoencoder.eval()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)  # Move data to the GPU
        reconstructed = autoencoder(data.unsqueeze(1).to(device))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Function to visualize 3D voxel data
def plot_voxel(voxel, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(voxel, facecolors='b', edgecolor='k')
    ax.set_title(title)
    plt.show()

# Visualize the original and reconstructed voxel grids
for i in range(len(test_data)):
    original_voxel = test_data[i].reshape(64, 64, 64)
    plot_voxel(original_voxel, "Original Voxel Grid")

    reconstructed_voxel = reconstructed[i].squeeze(0).cpu().numpy().reshape(64, 64, 64)
    plot_voxel(reconstructed_voxel, "Reconstructed Voxel Grid")
